# Exercise 2.0 Georeferenced plots - cartopy: Introduction and scatterplots

prepared by Mathias Hauser

The main library to plot georeferenced data/ map plots in python is [cartopy](http://scitools.org.uk/cartopy/). It is tightly coupled to matplotlib, so the general syntax is similar.

## Goals

* Display georeferenced data: create a map plot
* Get to know cartopy and the `projection` and `transform` keywords

## Import standard libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

We also need to import the coordinate reference system (crs) from cartopy, which is commonly abbreviated as `ccrs`:

In [ ]:
import cartopy.crs as ccrs

## Load Data

### Argo floats

As an example dataset, we use the position and temperature of [argo floats](http://www.argo.ucsd.edu/) for one day. Argo is a fleet of robotic instruments that drift with ocean currents and measure temperature, salinity, etc. The data was obtained from [ifremer](http://wwz.ifremer.fr/) (Institut français de recherche pour l'exploitation de la mer). The relevant data was extracted from the raw file in another notebook.

There are three separate files for the Atlantic, Indian, and Pacific Oceans, we combine all three datasets into one using `xr.concat`:

In [ ]:
date = "20171230"

fN = f"../data/ARGO_ATL_{date}.nc"
atl = xr.open_dataset(fN)

fN = f"../data/ARGO_IND_{date}.nc"
ind = xr.open_dataset(fN)

fN = f"../data/ARGO_PAC_{date}.nc"
pac = xr.open_dataset(fN)

argo = xr.concat([atl, ind, pac], dim="N_PROF")

In [ ]:
argo

The file contains the position (`lat` and `lon`)  and the ocean surface temperature (`TEMP`) of the floats for one day.

## First map

To plot a map, we have to specify a `projection` for the axes. We start with the `PlateCarree` projection, which is an [equirectangular projection](https://en.wikipedia.org/wiki/Equirectangular_projection). This projection maps `x` to be the value of the longitude and `y` to be the value of the latitude.

This creates a special kind of axes (a `<GeoAxesSubplot>`) which is derived from the normal `<AxesSubplot>`. It includes some special methods, one of which is called `ax.coastlines()` which allows to add coastlines.

Ok, let's get started:

In [ ]:
# notice the `()`:
projection = ccrs.PlateCarree()

f, ax = plt.subplots(subplot_kw=dict(projection=projection))

ax.coastlines()

# for non-maps this would be: `<AxesSubplot>`
ax

We created our first map plot!

There is a second way to create an axes with a certain projection:

```python
ax = plt.axes(projection=ccrs.PlateCarree())
```

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

While this is less to write, it does not allow for creating several subplots at once. We'll mostly use `plt.subplots` for the rest of the course, but both forms are fine.

### Exercise

* Create a figure with two subplots and add coastlines.

In [ ]:
# code here

### Solution

In [ ]:
f, axs = plt.subplots(1, 2, subplot_kw=dict(projection=projection))

axs[0].coastlines()
axs[1].coastlines()

#### Note

 > Because map plots have a fixed aspect ratio it's difficult to get a nice layout with `<GeoAxesSubplot>`. This is discussed in [Exercise 2.7](ex2_7_subplots.ipynb).

## Scatter plot - adding the position of the argo floats

### Exercise

 * Add the position of the argo floats with `ax.scatter` for all three ocean basins (`argo.lon`, `argo.lat`).
 
> this restricts the plot area to the position of the floats

 * Restore the global view with `ax.set_global()`

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()

# code here

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()

# code here
ax.scatter(argo.lon, argo.lat)

ax.set_global()

## Projections

The projection argument is used when creating plots and determines the kind of map of the plot (i.e. what the plot looks like). `cartopy` offers [different projections](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html). Check them out.

### Exercise
 * Redo the plot but use a `Robinson` projection.

In [ ]:
# update code

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()

h = ax.scatter(argo.lon, argo.lat)

ax.set_global()

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.scatter(argo.lon, argo.lat)

ax.set_global()

## `transform`

Something went wrong, the points are all at `lon=0`, `lat=0`! This is because now the map no longer has longitude and latitude coordinates. We need to tell the plotting function that we are passing lat/ lon data. We do that by passing in a coordinate system with the `transform` keyword:  This enables the geo axes to reproject the plot into the display projection.

### Exercise

* Add the `PlateCarree` `transform`ation to `ax.scatter` .

In [ ]:
# update code

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.scatter(argo.lon, argo.lat)

ax.set_global()

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

h = ax.scatter(argo.lon, argo.lat, transform=ccrs.PlateCarree())

ax.set_global()

Nice, we got our floats back... The takeaway from here is to **always** set the `transform` keyword for a map plot!

## Mapping colors

The dataset `argo` also contains the measured surface temperature. We want each displayed point to have a color according to its temperature.

### Exercise

* Color the points according to their temperature (`argo.TEMP`).
> This can be done with the `c=` argument of the scatter method.

In [ ]:
# update code

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

ax.scatter(argo.lon, argo.lat, transform=ccrs.PlateCarree())

ax.set_global()

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

ax.scatter(argo.lon, argo.lat, c=argo.TEMP, transform=ccrs.PlateCarree())

ax.set_global()

## Colorbar

In the plot above you don't know which color corresponds to which temperature - therefore we need to add a colorbar.

### Exercise

* Add a colorbar to know which color corresponds to which temperature.
* Restrict the range of the colormap to 0...25 using the `vmin` and `vmax` keywords.

In [ ]:
# update code

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = ax.scatter(argo.lon, argo.lat, transform=ccrs.PlateCarree(), c=argo.TEMP)

ax.set_global()

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = ax.scatter(
    argo.lon, argo.lat, transform=ccrs.PlateCarree(), c=argo.TEMP, vmin=0, vmax=25
)

ax.set_global()

plt.colorbar(h)

#### Note

 > The colorbar is too big for the map (also because the aspect ratio of the map is fixed). However, this is not straightforward to correct... In [Exercise 2.4](ex2_4_colorbars.ipynb) we will learn how to fix this.

## Colormaps

As already discussed, the chosen colors are according to the default colormap of matplotlib, "viridis". Colormaps can be set using the `cmap` keyword argument.

### Exercise

* Use the colormap named `"Reds"`.


In [ ]:
# update code

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = ax.scatter(
    argo.lon, argo.lat, transform=ccrs.PlateCarree(), c=argo.TEMP, vmin=0, vmax=25
)

plt.colorbar(h)

ax.set_global()

### Solution

In [ ]:
# solution

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = ax.scatter(
    argo.lon,
    argo.lat,
    transform=ccrs.PlateCarree(),
    c=argo.TEMP,
    vmin=0,
    vmax=25,
    cmap="Reds",
)

plt.colorbar(h)

ax.set_global()

## Set extent of maps

Until now we have not restricted the extent of the plots. While using `set_xlim` and `set_ylim` works for some projections (e.g. PlateCarree), they will fail in other cases. Therefore it is recommended to use `set_extent`: 

```python
ax.set_extent([lon_min, lon_max, lat_min, lat_max], ccrs.PlateCarree())
```
 > Adding `ccrs.PlateCarree()` (i.e. the coordinate reference system) to the function is required. Else your limits can be off.
 
### Exercise

* Restrict the plot to the Indian Ocean (e.g. 20°E to 150°E and 30°N to -75°N)

In [ ]:
# update code

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = ax.scatter(
    argo.lon, argo.lat, transform=ccrs.PlateCarree(), c=argo.TEMP, vmin=0, vmax=25
)

plt.colorbar(h)

ax.set_global()

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

h = ax.scatter(
    argo.lon, argo.lat, transform=ccrs.PlateCarree(), c=argo.TEMP, vmin=0, vmax=25
)

plt.colorbar(h)

ax.set_extent([20, 150, 30, -75], ccrs.PlateCarree())

## Bonus Material

* More features (e.g. lakes)
* Regional maps (e.g. Switzerland)
* Great circle lines

### Features & Natural Earth Data

`cartopy` can make use of many online geographical data sources. See examples under [Web Services](https://scitools.org.uk/cartopy/docs/latest/gallery/index.html). The best thing about this is that you have access and easy ways to plot all the data available at [naturalearthdata.com](http://www.naturalearthdata.com/). Naturalearth provides high-quality geo data for free. For example `ax.coastline()` displays the following data: [110m-coastline](http://www.naturalearthdata.com/downloads/110m-physical-vectors/110m-coastline/).

Note:

> The data is downloaded and stored the first time you use it, so this might take a moment.

In [ ]:
import cartopy.feature as cfeature

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.set_extent([-135, -50, 15, 55], ccrs.PlateCarree())

# ==========================================
# NaturalEarthData

ax.coastlines(resolution="50m", color="k")

# some data is easily accessible
ax.add_feature(cfeature.LAKES, edgecolor="0.1", zorder=100)

# for some data you need to know the name (e.g. the States of the US and Canada):
states = cfeature.NaturalEarthFeature(
    "cultural", "admin_1_states_provinces_lakes", "50m"
)
ax.add_feature(states, edgecolor="#b15928", facecolor="none", lw=0.5)

### Local maps

Plotting a map of Switzerland does not work differently than for the globe.

Note
> The outline of Switzerland was also obtained from NaturalEarthData - see [stack overflow](https://stackoverflow.com/a/47885128) or the [data preparation notebook](../data/prepare_data_MCH.ipynb).

In [ ]:
# load outline of switzerland

fN = "../data/outline_switzerland.nc"
ch = xr.open_dataset(fN)

This is the same plot as in [Exercise 1.3](../Part1_Matplotlib/ex1_3_scatter_plot.ipynb). The only two things that changed are the `projection` and `transform` keywords.

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.plot(ch.lon, ch.lat, transform=ccrs.PlateCarree())

plt.colorbar(h)

## A better projection for Switzerland

This looks terrible - but that's how Switzerland looks in the PlateCarree projection! Let's change the projection - find one that works for regional maps -> [cartopy projections](http://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html).

### Exercises

 * Try out `EuroPP`
 * Try out `LambertConformal` (with `central_longitude=15`)

In [ ]:
projection = ccrs.PlateCarree()

f, ax = plt.subplots(subplot_kw=dict(projection=projection))

ax.plot(ch.lon, ch.lat, transform=ccrs.PlateCarree())

plt.colorbar(h)

### Solution

In [ ]:
# solution

projection = ccrs.LambertConformal(central_longitude=15)

f, ax = plt.subplots(subplot_kw=dict(projection=projection))

ax.plot(ch.lon, ch.lat, transform=ccrs.PlateCarree())

plt.colorbar(h)

### Climatological Station Data for Switzerland - Temperature & Precip

Thus, we can repeat the plot we developed in [Exercise 1.3](../Part1_Matplotlib/ex1_3_scatter_plot.ipynb) on a proper map.

In [ ]:
# load climatological station data

fN = "../data/MCH_clim.nc"
clim = xr.open_dataset(fN)

# scale the precipitation to the point size
mn = clim.prec.min()
mx = clim.prec.max()

p_scaled = ((clim.prec - mn) / (mx - mn)) * 200 + 50

In [ ]:
projection = ccrs.LambertConformal(central_longitude=15)

ax = plt.axes(projection=projection)

ax.plot(ch.lon.values, ch.lat.values, transform=ccrs.PlateCarree())
h = ax.scatter(
    clim.lon.values,
    clim.lat.values,
    c=clim.temp.values,
    cmap="RdBu_r",
    vmax=8,
    vmin=-8,
    s=p_scaled,
    edgecolor="0.5",
    transform=ccrs.PlateCarree(),
    zorder=3,
)

plt.colorbar(h, label="Temperature [°C]")

for area in [1000, 1500, 2000]:
    size = ((area - mn) / (mx - mn)) * 200 + 50

    # convert number to string
    label = f"{area}"

    plt.scatter(
        [],
        [],
        c="0.85",
        s=size,
        label=label,
        edgecolor="0.5",
        transform=ccrs.PlateCarree(),
    )


plt.legend(
    title="Precipitation [mm / yr]", loc="upper center", ncol=3, edgecolor="none"
)

ax.set_extent((5.49, 10.77, 45.7, 48.3), ccrs.PlateCarree())


### Great circle lines

When plotting a line on a map with `plt.plot(..., transform=ccrs.PlateCarree())` this creates a straight line between the poins and not a great circle line. To show the shorthest path between two points on a sphere (i.e. a great circle line) we need to set `transform=ccrs.Geodetic()`.

Let's fly from Zürich (ZRH) to Vancover (YVR):

In [ ]:
# define location of airports

ZRH = (47.458361, 8.555264)
YVR = (49.196817, -123.180332)

lat = [ZRH[0], YVR[0]]
lon = [ZRH[1], YVR[1]]

# ==========================================

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

# direct line
ax.plot(lon, lat, transform=ccrs.PlateCarree(), color="#ff7f00")

# great circle line
ax.plot(lon, lat, transform=ccrs.Geodetic(), color="#e31a1c", marker="o")

# see comment
ax.set_extent([-135, 20, 10, 90], ccrs.PlateCarree())

# ==========================================

# add labels for airports

textopt = dict(
    transform=ccrs.PlateCarree(),
    ha="center",
    va="top",
    bbox=dict(facecolor="w", edgecolor="none", alpha=0.7),
)

ax.annotate("ZRH", xy=ZRH[::-1], xytext=(0, -10), textcoords="offset points", **textopt)
ax.annotate("YVR", xy=YVR[::-1], xytext=(0, -10), textcoords="offset points", **textopt)

# adding a slightly textured image of the globe
ax.stock_img();

### Side note: Fixing the bumpy great circle line

When you look closely, you can see that the great circle has some bumps in it. We can display it with higher accuracy.


In [ ]:
PC = ccrs.PlateCarree()
PC.threshold

In [ ]:
PC = ccrs.PlateCarree()

PC.threshold = 0.01

ax = plt.axes(projection=PC)

ax.coastlines()

# great circle line
ax.plot(lon, lat, transform=ccrs.Geodetic(), color="#e31a1c", marker="o")

# set extent
ax.set_extent([-135, 20, 10, 90], ccrs.PlateCarree())

# adding a slightly textured image of the globe
ax.stock_img();